First we must import numpy and SINDy

In [1]:

import numpy as np
import pandas as pd
from pysindy import SINDy

data = pd.read_csv('../ReducedTorqueData.csv')
data.head()

,timeInS,i_aInA,i_bInA,i_cInA,u_aInV,u_bInV,u_cInV,epsilon_elInRad
0,0.000000,1.765625,3.453125,0.937500,-0.322917,-0.677083,1.000000,-2.403804
1,0.000001,1.578125,3.421875,0.859375,-0.177083,-0.750000,0.927083,-2.403765
2,0.000002,1.578125,3.562500,0.750000,-0.114583,-0.854167,0.968750,-2.403726
3,0.000003,1.765625,3.671875,0.640625,-0.104167,-0.822917,0.927083,-2.403687
4,0.000004,1.656250,3.671875,0.609375,-0.125000,-0.802083,0.927083,-2.403648


In [ ]:
def main():
    # Define your data
    t = np.linspace(0, 10, 100)
    x = np.sin(t)
    y = np.cos(t)

    # Fit the model
    model = SINDy()
    model.fit([x, y], t=t)

    # Print the model coefficients
    print(model.coefficients())